# Molecular Dynamics with NequIP 

### Authors: Simon Batzner, Albert Musaelian, Lixin Sun, Anders Johansson, Boris Kozinsky

In [ ]:
!pip install torch==1.10

In [1]:
import warnings
import os

USE_COLAB = True
if USE_COLAB == True:
    from google.colab import drive
    drive.mount('/content/drive')
    work_dir = '/content/drive/MyDrive/Colab Notebooks/nequip/'
    data_dir = '/content/'
else:
    work_dir = '/Users/gabrieletocci/Google Drive/My Drive/Colab Notebooks/nequip/'
    data_dir = '/Users/gabrieletocci/Documents/projects/MD_DFT/'


Mounted at /content/drive


In [2]:
import torch
print(torch. __version__)
torch.cuda.is_available()

1.10.0+cu102


True

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [4]:
!rm -r lammps
!git clone -b "stable_29Sep2021_update2" --depth 1 "https://github.com/lammps/lammps.git"
!git clone https://github.com/mir-group/pair_nequip
!cd pair_nequip && ./patch_lammps.sh /content/lammps/
!cd ..

Cloning into 'lammps'...
remote: Enumerating objects: 11732, done.
remote: Counting objects: 100% (11732/11732), done.
remote: Compressing objects: 100% (8605/8605), done.
remote: Total 11732 (delta 3940), reused 6322 (delta 2928), pack-reused 0
Receiving objects: 100% (11732/11732), 110.00 MiB | 18.02 MiB/s, done.
Resolving deltas: 100% (3940/3940), done.
Note: checking out '7586adbb6a61254125992709ef2fda9134cfca6c'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

Checking out files: 100% (11058/11058), done.
fatal: destination path 'pair_nequip' already exists and is not an empty directory.
Copying files...
Updating CMakeLists.txt...


In [5]:
!cp /content/pair_nequip/*.cpp /content/lammps/src/
!cp /content/pair_nequip/*.h /content/lammps/src/
! sed -i 's/CMAKE_CXX_STANDARD 11/CMAKE_CXX_STANDARD 14/g'  /content/lammps/cmake/CMakeLists.txt

In [6]:
!pip install mkl mkl-include

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!cd lammps && mkdir -p build && cd build && cmake ../cmake -DCMAKE_PREFIX_PATH=`python -c 'import torch;print(torch.utils.cmake_prefix_path)'` && make -j4

-- The CXX compiler identification is GNU 7.5.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Git: /usr/bin/git (found version "2.17.1") 
-- Appending /usr/local/cuda/lib64/stubs to CMAKE_LIBRARY_PATH: /usr/local/cuda/lib64/stubs
-- Running check for auto-generated files from make-based build system
-- Found MPI_CXX: /usr/lib/x86_64-linux-gnu/openmpi/lib/libmpi_cxx.so (found version "3.1") 
-- Found MPI: TRUE (found version "3.1")  
-- Looking for C++ include omp.h
-- Looking for C++ include omp.h - found
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- Found JPEG: /usr/lib/x86_64-linux-gnu/libjpeg.so (found version "80") 
-- Found PNG: /usr/lib/x86_64-linux-gnu/libpng.so (found version "1.6.34") 
-- Found ZLIB: /usr/lib/x86_64-linux-gnu/libz.so (found version "1.

In [8]:
# Uncomment if running on google colab
! tar -xzvf  /content/drive/MyDrive/Colab\ Notebooks/MD_DFT/AIMD_data.tar.gz

._AIMD_data
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.google.drivefs.item-id#S'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.lastuseddate#PS'
AIMD_data/
AIMD_data/._WATER-frc-10k-1.xyz
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.google.drivefs.item-id#S'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.lastuseddate#PS'
AIMD_data/WATER-frc-10k-1.xyz
AIMD_data/._celldata.dat
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.google.drivefs.item-id#S'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.lastuseddate#PS'
AIMD_data/celldata.dat
AIMD_data/._WATER-pos-10k-1.xyz
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.google.drivefs.item-id#S'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.lastuseddate#PS'
AIMD_data/WATER-pos-10k-1.xyz


In [9]:
import numpy as np
import pandas as pd
# install wandb
!pip install wandb
# install nequip
#uncomment if running on google colab
!git clone --depth 1 "https://github.com/gabriele16/nequip.git"
!pip install nequip/
# fix colab imports
import site
site.main()
# set to allow anonymous WandB
import os
os.environ["WANDB_ANONYMOUS"] = "must"
import numpy as np
from ase.io import read, write
np.random.seed(0)
torch.manual_seed(0)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 7.1 MB/s 
     |████████████████████████████████| 181 kB 56.2 MB/s 
     |████████████████████████████████| 145 kB 50.8 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=352574ac23c6e032d0aa9c9b0afd65b27f1b0ae51fd98e5f6b5e0e8f4c448c54
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
Cloning into 'nequip'...
remote: Enumerating objects: 183, done.
remote: Counting objects: 100% (183/183), done.
remote: Compressing objects: 100% (178/178), done.
remote: Total 183 (delta 8), reused 87 (delta 0), pack-reused 0
Receiving objects: 100% (183/183), 333.98 KiB | 15.18 MiB/s, done.
Resolving deltas: 100% (8/8), done.
Looking in indexes: https://pypi.org/simple, https

In [10]:
def MD_reader_xyz(f, data_dir, no_skip = 0):
  filename = os.path.join(data_dir, f)
  fo = open(filename, 'r')
  natoms_str = fo.read().rsplit(' i = ')[0]
  natoms = int(natoms_str.split('\n')[0])
  fo.close()  
  fo = open(filename, 'r')
  samples = fo.read().split(natoms_str)[1:]
  steps = []
  xyz = []
  temperatures = []
  energies = []
  for sample in samples[::no_skip]:
     entries = sample.split('\n')[:-1]
     energies.append(float(entries[0].split("=")[-1]))
     temp = np.array([list(map(float, lv.split()[1:])) for lv in entries[1:]])
     xyz.append(temp[:,:])
  return natoms_str, np.array(xyz), np.array(energies)     

In [50]:
def MD_writer_xyz(positions,forces,cell_vec_abc,energies,
                  data_dir,f,  conv_frc = 1.0 , conv_ener = 1.0 ):

  filename = os.path.join(data_dir, f)
  fo = open(filename, 'w')

  for it, frame in enumerate(positions):
    natoms = len(frame)
    fo.write("{:5d}\n".format(natoms))
    fo.write('Lattice="{:.5f} 0.0 0.0 0.0 {:.5f} 0.0 0.0 0.0 {:.5f}" \
    Properties="species:S:1:pos:R:3:forces:R:3" \
    energy={:.10f} pbc="T T T"\n'.format(cell_vec_abc[0],cell_vec_abc[1],cell_vec_abc[2],energies[it]*conv_ener)    
    )
    if it%100 == 0.0:
      print(it)
    force_frame = forces[it]

    fo.write("".join("{:8s} {:.8f} {:16.8f} {:16.8f}\
     {:16.8f} {:16.8f} {:16.8f}\n".format(frame[iat].symbol,
                                          frame[iat].position[0],
                                          frame[iat].position[1],
                                          frame[iat].position[2],
                                          force_frame[iat].position[0]*conv_frc,
                                          force_frame[iat].position[1]*conv_frc,
                                          force_frame[iat].position[2]*conv_frc)
                                          for iat in range(len(frame))))

In [12]:
def read_cell(f,data_dir):
  filename = os.path.join(data_dir,f)
  fo = open(filename,'r')
  cell_list_abc = fo.read().split('\n')[:-1]
  cell_vec_abc = np.array([list(map(float, lv.split())) for lv in cell_list_abc]).squeeze()
  return(cell_vec_abc)

cell_vec_abc = read_cell('celldata.dat',data_dir + '/AIMD_data')
cell_vec_abc

array([9.85, 9.85, 9.85])

In [13]:
wat_traj = read(data_dir +'AIMD_data/WATER-pos-10k-1.xyz',index=':')
wat_frc = read(data_dir + 'AIMD_data/WATER-frc-10k-1.xyz', index=':')

In [14]:
natoms, positions, energies = MD_reader_xyz('WATER-pos-10k-1.xyz', data_dir + '/AIMD_data/', no_skip=1)

In [15]:
MD_writer_xyz(wat_traj, wat_frc, cell_vec_abc, energies, data_dir + '/AIMD_data/', 'wat_pos_frc-10k.extxyz',conv_frc = 1.0, conv_ener = 27.211399)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


### Turn on GPU

Make sure Runtime --> Change runtime type is set to GPU

## 3 Steps: 
* Train: using a data set, train the neural network 🧠 
* Deploy: convert the Python-based model into a stand-alone potential file for fast execution ⚡
* Run: run Molecular Dynamics, Monte Carlo, Structural Minimization, ...  with it in LAMMPS 🏃

<img src="https://github.com/mir-group/nequip_mrs_tutorial/blob/master/all.png?raw=true" width="60%">

### Train a model

<img src="https://github.com/mir-group/nequip_mrs_tutorial/blob/master/train.png?raw=true" width="60%">

This tutorial is set up to use `wandb` in anonymous mode; when you use NequIP yourself you will be presented with a login prompt.

Here, we will train a NequIP potential on the following system

* Toluene
* sampled at T=500K from AIMD
* at CCSD(T) accuracy (gold standard of quantum chemistry)
* Using 100 training configurations
* The units of the reference data are in kcal/mol and A. If you're more familiar with eV, remember 1 kcal/mol is chemical accuracy and is approximately 43 meV

Start a training run: this will print output to our console, but it is usually more convenient to view the results in a web interface called Weights and Biases. Click the link next to the rocket emoji to watch the run in the WandB interface 🚀 

In WandB, watch the followingkeys:

* Plot 1: validation_all_f_mae, training_all_f_mae
* Plot 2: validation_e/N_mae, training_e/N_mae

These are the validation/training error in all force components and the validation/training error in the potential energy, normalized by the number of atoms, respectively. 

In [16]:
from nequip.utils import Config
config = Config.from_file(work_dir + 'configs/my-full-example.yaml')
config

{'root': 'results/water', 'run_name': 'example-run-water', 'seed': 123, 'dataset_seed': 456, 'append': True, 'default_dtype': 'float32', 'allow_tf32': False, 'r_max': 4.0, 'num_layers': 4, 'l_max': 1, 'parity': True, 'num_features': 32, 'nonlinearity_type': 'gate', 'resnet': False, 'nonlinearity_scalars': {'e': 'silu', 'o': 'tanh'}, 'nonlinearity_gates': {'e': 'silu', 'o': 'tanh'}, 'num_basis': 8, 'BesselBasis_trainable': True, 'PolynomialCutoff_p': 6, 'invariant_layers': 2, 'invariant_neurons': 64, 'avg_num_neighbors': 'auto', 'use_sc': True, 'dataset': 'ase', 'dataset_file_name': './AIMD_data/wat_pos_frc-10k.extxyz', 'ase_args': {'format': 'extxyz'}, 'include_keys': ['user_label'], 'key_mapping': {'user_label': 'label0'}, 'chemical_symbols': ['H', 'O'], 'verbose': 'info', 'log_batch_freq': 1, 'log_epoch_freq': 1, 'save_checkpoint_freq': -1, 'save_ema_checkpoint_freq': -1, 'n_train': 100, 'n_val': 50, 'learning_rate': 0.005, 'batch_size': 5, 'validation_batch_size': 10, 'max_epochs': 

In [18]:
!ls /content/drive/MyDrive/Colab\ \Notebooks/nequip/configs/my-full-example.yaml
!rm -rf ./results
!nequip-train /content/drive/MyDrive/Colab\ \Notebooks/nequip/configs/my-full-example.yaml

'/content/drive/MyDrive/Colab Notebooks/nequip/configs/my-full-example.yaml'
Torch device: cuda
Processing dataset...
Loaded data: Batch(atomic_numbers=[960000, 1], batch=[960000], cell=[10000, 3, 3], edge_cell_shift=[25556838, 3], edge_index=[2, 25556838], forces=[960000, 3], pbc=[10000, 3], pos=[960000, 3], ptr=[10001], total_energy=[10000, 1])
Cached processed data to disk
Done!
Successfully loaded the data set of type ASEDataset(10000)...
Replace string dataset_forces_rms to 0.7736424803733826
Replace string dataset_per_atom_total_energy_mean to -5.736269474029541
Atomic outputs are scaled by: [H, O: 0.773642], shifted by [H, O: -5.736269].
Replace string dataset_forces_rms to 0.7736424803733826
Initially outputs are globally scaled by: 0.7736424803733826, total_energy are globally shifted by None.
Successfully built the network...
Number of weights: 154200
! Starting training ...

validation
# Epoch batch         loss       loss_f       loss_e        f_mae       f_rmse      H_f_ma

We see that the model has converged to an energy accuarcy < 1meV/atom and a force accuracy of approx. 40 meV/A within 5 minutes and trained on only 100 samples. That should give us a good first potential! Note that these numbers will decrease significantly if you increase the training set size and the number of epochs to train. 

### Deploy the model

<img src="https://github.com/mir-group/nequip_mrs_tutorial/blob/master/deploy.png?raw=true" width="60%">

We now convert the model to a potential file. This makes it independent of NequIP and we can use it any downstream application, such as LAMMPS. 

In [20]:
! ls -lrth results/water/example-run-water

total 5.3M
-rw------- 1 root root 227K Jul  5 18:38 config.yaml
-rw-r--r-- 1 root root  489 Jul  5 18:38 metrics_initialization.csv
-rw-r--r-- 1 root root 517K Jul  5 18:43 metrics_batch_train.csv
-rw-r--r-- 1 root root 131K Jul  5 18:43 metrics_batch_val.csv
-rw-r--r-- 1 root root  45K Jul  5 18:43 metrics_epoch.csv
-rw------- 1 root root 657K Jul  5 18:43 best_model.pth
-rw-r--r-- 1 root root 594K Jul  5 18:43 log
-rw------- 1 root root 2.5M Jul  5 18:43 trainer.pth
-rw------- 1 root root 657K Jul  5 18:43 last_model.pth


In [44]:
! nequip-deploy -h

usage: nequip-deploy [-h] [--verbose VERBOSE] {info,build} ...

Create and view information about deployed NequIP potentials.

optional arguments:
  -h, --help         show this help message and exit
  --verbose VERBOSE  log level

commands:
  {info,build}
    info             Get information from a deployed model file
    build            Build a deployment model


In [26]:
!nequip-deploy build --train-dir /content/results/water/example-run-water water-deploy.pth

INFO:root:Loading best_model from training session...
INFO:root:Compiled & optimized model.


## Evaluate Test Error on all remaining frames

Before running inference, we'd like to know how well the model is doing on a hold-out test set. We run the nequip-evaluate command to compute the test error on all data that we didn't use for training or validation. 

In [27]:
!nequip-evaluate --train-dir results/water/example-run-water --batch-size 50

Using device: cuda
Loading model... 
loaded model from training session
Loading original dataset...
Loaded dataset specified in config.yaml.
Using origial training dataset (10000 frames) minus training (100 frames) and validation frames (50 frames), yielding a test set size of 9850 frames.
Starting...
  0% 0/9850 [00:00<?, ?it/s]

  1% 50/9850 [00:00<01:15, 129.57it/s]
  1% 100/9850 [00:01<02:07, 76.54it/s]
  2% 150/9850 [00:02<03:15, 49.60it/s]
  2% 200/9850 [00:04<03:44, 42.93it/s]
  3% 250/9850 [00:04<02:39, 60.11it/s]
  3% 300/9850 [00:04<02:00, 79.17it/s]
  4% 350/9850 [00:04<01:36, 98.85it/s]
  4% 400/9850 [00:05<01:19, 118.14it/s]
  5% 450/9850 [00:05<01:08, 136.32it/s]
  5% 500/9850 [00:05<01:01, 151.77it/s]
  6% 550/9850 [00:05<00:56, 164.34it/s]
  6% 600/9850 [00:06<00:53, 174.33it/s]
  7% 650/9850 [00:06<00:50, 182.06it/s]
  7% 700/9850 [00:06<00:48, 187.54it/s]
  8% 750/9850 [00:06<00:47, 192.12it/s]
  8% 800/9850 [00:07<00:46, 195.41it/s]
  9% 850/9850 [00:07<00:45, 198.47

Again, energy errors of < 1meV/atom (converted from kcal/mol to eV), and force errors of ~45 meV/A 🎉

# LAMMPS

We are now in a position to run MD with our potential. Here, we will minimize the geometry of the toluene molecule we trained on from a perturbed initial state. 

<img src="https://github.com/mir-group/nequip_mrs_tutorial/blob/master/run.png?raw=true" width="60%">

Set up a simple LAMMPS input file

CAUTION: the reference data here are in kcal/mol for the energies and kcal/mol/A for the forces. The NequIP model will therefore also be predicting outputs in these units. We are therefore using `units real` in LAMMPS (see [docs](https://docs.lammps.org/units.html)). If your reference data are in other units, you should using the corresponding units command in LAMMPS (e.g. if you use eV, A then `units metal` would be appropriate, which would then also change time units from `fs` to `ps`).

In [51]:
lammps_input_md = """
units           metal
boundary        p p p
atom_style      atomic

neighbor        1.0 bin
neigh_modify    every 10 delay 0 check no

pair_style	nequip
pair_coeff	* * ../water-deploy.pth O H 
mass            1 15.9994
mass            2 1.00794

velocity        all create 330.0 23456789

fix             1 all nvt temp 330.0 330.0 1.0
timestep        0.001
thermo_style    custom step pe ke etotal temp press vol
thermo          100

dump            1 all custom 100 water_sheets.dump id type x y z
dump            2 all custom 1 dump_frc.lamppstrj id type element fx fy fz
dump_modify     2 element O H
dump            3 all custom 1 dump.lammpstrj id type element x y z
dump_modify     3 element O H

run             10000
"""

!mkdir lammps_run
with open("lammps_run/water_md.in", "w") as f:
    f.write(lammps_input_md)

mkdir: cannot create directory ‘lammps_run’: File exists


In [31]:
lammps_input_minimize = """
units	real
atom_style atomic
newton off
thermo 1
read_data structure.data

pair_style	nequip
pair_coeff	* * ../water-deploy.pth O H 
mass            1 15.9994
mass            2 1.00794

neighbor 1.0 bin
neigh_modify delay 5 every 1

minimize 0.0 1.0e-8 10000 1000000
write_dump all custom output.dump id type x y z fx fy fz
"""
!mkdir lammps_run
with open("lammps_run/water_minimize.in", "w") as f:
    f.write(lammps_input_minimize)

mkdir: cannot create directory ‘lammps_run’: File exists


In [29]:
! cp /content/water-deploy.pth /content/lammps_run/.

Here's starting configuration for Toluene at CCSD(T) accuracy. We will strongly perturb the inital positions by sampling from a uniform distribution $\mathcal{U}([0, 0.5])$

In [ ]:
# toluene_example = """15
#  Lattice="100.0 0.0 0.0 0.0 100.0 0.0 0.0 0.0 100.0" Properties=species:S:1:pos:R:3 -169777.5840406276=T pbc="F F F"
#  C       52.48936904      49.86911725      50.09520748
#  C       51.01088202      49.89609925      50.17978049
#  C       50.36647401      50.04650925      48.96054247
#  C       48.95673398      50.29576626      48.71580846
#  C       48.04533296      50.26023426      49.82589448
#  C       48.70932398      49.85770925      51.01923950
#  C       50.06326400      49.77782925      51.25691751
#  H       52.94467905      50.48672926      50.86545150
#  H       52.89060405      48.87175023      50.14480949
#  H       53.02173405      50.05890725      49.03968247
#  H       51.01439802      50.38234726      48.05314045
#  H       48.80598498      50.64314926      47.68195744
#  H       46.96754695      50.20586626      49.53998848
#  H       48.16716997      49.75850325      51.88622952
#  H       50.45791001      49.55387424      52.15303052
#  """

# with open('toluene.xyz', 'w') as f: 
#     f.write(toluene_example)

# # read as ASE objects
# atoms = read('toluene.xyz', format='extxyz')

# # perturb positions
# p = atoms.get_positions()
# p += np.random.rand(15, 3) * 0.5
# atoms.set_positions(p)
# atoms.set_pbc(False)

# # write to a LAMMPS file
# write("lammps_run/structure.data", atoms, format="lammps-data")

In [47]:
wat_pos_frc_trj = read('/content/AIMD_data/wat_pos_frc-10k.extxyz')
write("/content/lammps_run/structure.data", wat_pos_frc_trj,format='lammps-data')


### Run the LAMMPS command: 

In [48]:
cat /content/lammps_run/water_md.in


units	real
atom_style atomic
newton off
thermo 1
read_data structure.data

pair_style	nequip
pair_coeff	* * ../water-deploy.pth O H 
mass            1 15.9994
mass            2 1.00794

neighbor 1.0 bin
neigh_modify delay 5 every 1

minimize 0.0 1.0e-8 10000 1000000
write_dump all custom output.dump id type x y z fx fy fz


In [49]:
!cd /content/lammps_run/ && ../lammps/build/lmp -in water_md.in

LAMMPS (29 Sep 2021 - Update 2)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Reading data file ...
  orthogonal box = (0.0000000 0.0000000 0.0000000) to (9.8500000 9.8500000 9.8500000)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  96 atoms
  read_data CPU = 0.001 seconds
NEQUIP is using device cuda
NequIP Coeff: type 1 is element O
NequIP Coeff: type 2 is element H
Loading model from ../water-deploy.pth
Freezing TorchScript model...
Neighbor list info ...
  update every 1 steps, delay 0 steps, check yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 5
  ghost atom cutoff = 5
  binsize = 2.5, bins = 4 4 4
  1 neighbor lists, perpetual/occasional/extra = 1 0 0
  (1) pair nequip, perpetual
      attributes: full, newton off
      pair build: full/bin/atomonly
      stencil: full/bin/3d
      bin: standard
Setting up cg style minimization ...
  Unit style    : real
  Current

In [ ]:
!pip3 install nglview
import nglview as nv

In [ ]:
#proj1 = mda.Merge(backbone1)
#proj1.load_new(coordinatesu1, order="fac")

In [ ]:
#view = nv.show_mdanalysis(proj1.atoms)
#view

In [ ]:
# !pip install moviepy==0.2.2.11
# !pip install imageio==1.6

In [ ]:
# from nglview.contrib.movie import MovieMaker
# movie = MovieMaker(view, output='pc1u1.gif', in_memory=True)
# movie.make()

We see LAMMPS converges quickly to a minimum. Let's check how well we did. 

In [ ]:
# read the final structure back in 
#minimized = read('./lammps_run/output.dump', format='lammps-dump-text')

### Compare optimized bond length to true coupled cluster reference from CCCBDB

In [ ]:
# get distances of optimized geometry (reference data: CCSD(T) [Psi4, cc-pVDZ])
# d_12 = minimized.get_distances(1, 2)

# # reference: https://cccbdb.nist.gov/geom3x.asp?method=6&basis=2, coupled cluster
# d_12_ccd = 1.4086

# print('Relative Error in bond length w.r.t. Coupled Cluster from CCCBDB: {:.3f}%'.format((100 * np.abs(d_12 - d_12_ccd) / d_12_ccd)[0]))

Relative Error in bond length w.r.t. Coupled Cluster from CCCBDB: 0.100%


We find a final relative error close to Coupled Cluster accuracy 🎉

## Next Steps

This concludes our tutorial. A next step would be to head over to https://github.com/mir-group/nequip, install NequIP and get started with your own system. If you have questions, please don't hesitate to reach out to batzner@g.harvard.edu, we're happy to help! 

